### Market Share Analysis using data from the [Casablanca-bourse](https://www.casablanca-bourse.com/fr/live-market/emetteurs/AFM151215) Website



> **Web Scraping part**

**By:**

* [Hamza Khalid](https://hamzakhalid.me/)
* [Ismail Ouahbi](https://ismailouahbi.github.io/)


---



#### Scraping part [Considering date filters & Holidays + Weekends]

In [ ]:
# import libraries
# import packages
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from datetime import datetime, timedelta
import calendar
import holidays
from babel.dates import format_date

# Define your driver

# define your path to chrome driver
chrome_driver = "link to your chrome driver"

# Set the path to the ChromeDriver executable
webdriver_service = Service(chrome_driver)

# Set any additional options for the ChromeDriver (if required)
webdriver_options = webdriver.ChromeOptions()
# webdriver_options.add_argument('--headless')  # Uncomment this line to run in headless mode

# Create a new instance of the Chrome driver
driver = webdriver.Chrome(service=webdriver_service, options=webdriver_options)

In [ ]:
# define site url
url = 'https://www.casablanca-bourse.com/fr/composition-des-indices'
driver.get(url)

##### Generate a list of dates from 26/12/23 to 23/06/23 in the form of (décembre 26, 2022)

In [ ]:
# Define the start and end dates
start_date = datetime(2022, 12, 26)
end_date = datetime(2023, 6, 23)

# Define a list to store the months
months = []

# Generate the months
current_date = start_date
while current_date <= end_date:
    # Get the month name in French
    month_name_french = format_date(current_date, 'dd MMMM yyyy', locale='fr')

    # Add the month to the list
    months.append(month_name_french)

    # Move to the next month
    current_date += timedelta(days=1)

In [ ]:
# remove zero from the begining of some elements

for i in range(len(months)):
    if months[i].startswith('0'):
        months[i] = months[i][1:]

In [ ]:
# verify
months

['26 décembre 2022',
 '27 décembre 2022',
 '28 décembre 2022',
 '29 décembre 2022',
 '30 décembre 2022',
 '31 décembre 2022',
 '1 janvier 2023',
 '2 janvier 2023',
 '3 janvier 2023',
 '4 janvier 2023',
 '5 janvier 2023',
 '6 janvier 2023',
 '7 janvier 2023',
 '8 janvier 2023',
 '9 janvier 2023',
 '10 janvier 2023',
 '11 janvier 2023',
 '12 janvier 2023',
 '13 janvier 2023',
 '14 janvier 2023',
 '15 janvier 2023',
 '16 janvier 2023',
 '17 janvier 2023',
 '18 janvier 2023',
 '19 janvier 2023',
 '20 janvier 2023',
 '21 janvier 2023',
 '22 janvier 2023',
 '23 janvier 2023',
 '24 janvier 2023',
 '25 janvier 2023',
 '26 janvier 2023',
 '27 janvier 2023',
 '28 janvier 2023',
 '29 janvier 2023',
 '30 janvier 2023',
 '31 janvier 2023',
 '1 février 2023',
 '2 février 2023',
 '3 février 2023',
 '4 février 2023',
 '5 février 2023',
 '6 février 2023',
 '7 février 2023',
 '8 février 2023',
 '9 février 2023',
 '10 février 2023',
 '11 février 2023',
 '12 février 2023',
 '13 février 2023',
 '14 février

In [ ]:
# remove 24 avril 2023 as an exception

months.remove('24 avril 2023')

##### Generate list of weekends and holidays in morocco between 26/12/22 and 23/06/23

In [ ]:
# Create a list of holidays in Morocco
morocco_holidays = holidays.Morocco()

# Create a list of weekends and holidays
weekends_and_holidays = []

# Generate dates for a certain period
start_date = datetime(2022, 12, 26)
end_date = datetime(2023, 6, 23)
delta = timedelta(days=1)

while start_date <= end_date:
    # If the date is a weekend or a holiday, add it to the list
    if start_date.weekday() >= 5 or start_date in morocco_holidays:
        weekends_and_holidays.append(start_date)
    start_date += delta


# as the weekends_and_holidays is of type datetime object, we need to convert it to match the french notation

weekends_and_holidays = [format_date(date, 'dd MMMM yyyy', locale='fr') for date in weekends_and_holidays]

In [ ]:
# same process for weekends and holidays (remove zero from the start)

# remove zero from the begining of some elements

for i in range(len(weekends_and_holidays)):
    if weekends_and_holidays[i].startswith('0'):
        weekends_and_holidays[i] = weekends_and_holidays[i][1:]

### Scrape the data and considering date filter and pagination within each date

In [ ]:
## Write a function to do the whole process

def scrape_pages():
    # create the 'data' data frame to hold data
    table = driver.find_element(By.CSS_SELECTOR, 'table.w-full.print\\:max-w-full')
    # Extract the column names from the first row
    header_row = table.find_element(By.TAG_NAME, 'tr')
    column_names = [header.text for header in header_row.find_elements(By.TAG_NAME, 'th')]

    ##add also the date column
    column_names = column_names + ['date']

    # Create an empty dataframe with the extracted column names
    data = pd.DataFrame(columns=column_names)

    # Now scrape the data according to the conditions

    for date_to_scrape in months:
        if date_to_scrape not in weekends_and_holidays:
            print('-----------------------')
            print('date',date_to_scrape)
            # set input default value to this date
            js = f"""
            var inputField = document.querySelector('input[placeholder="Séance"]');
            var nativeInputValueSetter = Object.getOwnPropertyDescriptor(window.HTMLInputElement.prototype, "value").set;
            nativeInputValueSetter.call(inputField, "{date_to_scrape}");

            var ev2 = new Event('input', {{'bubbles': true}});
            inputField.dispatchEvent(ev2);
            """
            driver.execute_script(js)

            # click on submit button

            # Wait for the updated date value to be visible
            wait = WebDriverWait(driver, 150)
            wait.until(EC.text_to_be_present_in_element_value((By.XPATH, '//input[@placeholder="Séance"]'), date_to_scrape))

            # Find the confirm button by inner HTML text
            confirm_button = driver.find_element(By.XPATH, '//button[text()="Appliquer"]')

            # simulate a mouse click on this button to click on it
            action = ActionChains(driver)
            action.move_to_element(confirm_button).click().perform()

            # Scrape data for this date
                # Extract the table rows
            rows = table.find_elements(By.TAG_NAME, 'tr')

            # Scrape the data from each row and add it to the dataframe
            for row in rows[1:]:  # Skip the header row

                # Extract the table cells (columns) from each row
                cells = row.find_elements(By.TAG_NAME, 'td')

                # Extract the text from each cell
                row_data = [cell.text for cell in cells] + [date_to_scrape]

                # Add the row data to the dataframe
                data.loc[len(data)] = row_data

            # paginate
            # Find the next page button
            next_page_button = driver.find_element(By.CSS_SELECTOR, '.text-sm.leading-5.w-10.h-10.rounded-full.mx-1.pointer.ease-in.transition\\:all.duration-300.bg-transparent.text-gray-900.md\\:hover\\:bg-primary-500.md\\:hover\\:text-white')
            driver.execute_script("arguments[0].click();", next_page_button)
            # Add a small delay to ensure the content is loaded
            time.sleep(4)
            print('Button clicked')
            print('--'*10)

             # Find the updated table element
            table = driver.find_element(By.XPATH, '//table')
            print('tabel', table)
            print('--'*10)

            time.sleep(3)
            # Extract data from each row and append it to the dataframe
            rows = table.find_elements(By.TAG_NAME, 'tr')
            print('rows', rows)
            print('--'*10)
            # Scrape the data from each row and add it to the dataframe
            for row in rows[1:]:# Skip the header row
                print('in for loop')
                print('row', row)
                print('--'*10)
                # Extract the table cells (columns) from each row
                cells = row.find_elements(By.TAG_NAME, 'td')

                # Extract the text from each cell
                row_data = [cell.text for cell in cells] + [date_to_scrape]

                # Add the row data to the dataframe
                data.loc[len(data)] = row_data

                # save to csv file (for back-up)
                data.to_csv('data/data.csv' , index_label=False)
            print('----------------')
            print('data', data)

    return data

In [ ]:
## scrape all data

data = scrape_pages()

-----------------------
date 26 décembre 2022
Button clicked
--------------------
tabel <selenium.webdriver.remote.webelement.WebElement (session="76251786298407fb119989d6a661ba50", element="87182FA69B721DFDF43E1490D8667C8F_element_11")>
--------------------
rows [<selenium.webdriver.remote.webelement.WebElement (session="76251786298407fb119989d6a661ba50", element="87182FA69B721DFDF43E1490D8667C8F_element_12")>, <selenium.webdriver.remote.webelement.WebElement (session="76251786298407fb119989d6a661ba50", element="87182FA69B721DFDF43E1490D8667C8F_element_19")>, <selenium.webdriver.remote.webelement.WebElement (session="76251786298407fb119989d6a661ba50", element="87182FA69B721DFDF43E1490D8667C8F_element_20")>, <selenium.webdriver.remote.webelement.WebElement (session="76251786298407fb119989d6a661ba50", element="87182FA69B721DFDF43E1490D8667C8F_element_21")>, <selenium.webdriver.remote.webelement.WebElement (session="76251786298407fb119989d6a661ba50", element="87182FA69B721DFDF43E1490D866

in for loop
row <selenium.webdriver.remote.webelement.WebElement (session="76251786298407fb119989d6a661ba50", element="87182FA69B721DFDF43E1490D8667C8F_element_20")>
--------------------
in for loop
row <selenium.webdriver.remote.webelement.WebElement (session="76251786298407fb119989d6a661ba50", element="87182FA69B721DFDF43E1490D8667C8F_element_21")>
--------------------
in for loop
row <selenium.webdriver.remote.webelement.WebElement (session="76251786298407fb119989d6a661ba50", element="87182FA69B721DFDF43E1490D8667C8F_element_22")>
--------------------
in for loop
row <selenium.webdriver.remote.webelement.WebElement (session="76251786298407fb119989d6a661ba50", element="87182FA69B721DFDF43E1490D8667C8F_element_23")>
--------------------
in for loop
row <selenium.webdriver.remote.webelement.WebElement (session="76251786298407fb119989d6a661ba50", element="87182FA69B721DFDF43E1490D8667C8F_element_24")>
--------------------
in for loop
row <selenium.webdriver.remote.webelement.WebElement 

in for loop
row <selenium.webdriver.remote.webelement.WebElement (session="76251786298407fb119989d6a661ba50", element="87182FA69B721DFDF43E1490D8667C8F_element_395")>
--------------------
in for loop
row <selenium.webdriver.remote.webelement.WebElement (session="76251786298407fb119989d6a661ba50", element="87182FA69B721DFDF43E1490D8667C8F_element_396")>
--------------------
in for loop
row <selenium.webdriver.remote.webelement.WebElement (session="76251786298407fb119989d6a661ba50", element="87182FA69B721DFDF43E1490D8667C8F_element_397")>
--------------------
in for loop
row <selenium.webdriver.remote.webelement.WebElement (session="76251786298407fb119989d6a661ba50", element="87182FA69B721DFDF43E1490D8667C8F_element_398")>
--------------------
in for loop
row <selenium.webdriver.remote.webelement.WebElement (session="76251786298407fb119989d6a661ba50", element="87182FA69B721DFDF43E1490D8667C8F_element_399")>
--------------------
----------------
data    Ticker          Instrument Nombre d

in for loop
row <selenium.webdriver.remote.webelement.WebElement (session="76251786298407fb119989d6a661ba50", element="87182FA69B721DFDF43E1490D8667C8F_element_29")>
--------------------
in for loop
row <selenium.webdriver.remote.webelement.WebElement (session="76251786298407fb119989d6a661ba50", element="87182FA69B721DFDF43E1490D8667C8F_element_30")>
--------------------
in for loop
row <selenium.webdriver.remote.webelement.WebElement (session="76251786298407fb119989d6a661ba50", element="87182FA69B721DFDF43E1490D8667C8F_element_31")>
--------------------
in for loop
row <selenium.webdriver.remote.webelement.WebElement (session="76251786298407fb119989d6a661ba50", element="87182FA69B721DFDF43E1490D8667C8F_element_32")>
--------------------
in for loop
row <selenium.webdriver.remote.webelement.WebElement (session="76251786298407fb119989d6a661ba50", element="87182FA69B721DFDF43E1490D8667C8F_element_33")>
--------------------
in for loop
row <selenium.webdriver.remote.webelement.WebElement 

in for loop
row <selenium.webdriver.remote.webelement.WebElement (session="76251786298407fb119989d6a661ba50", element="87182FA69B721DFDF43E1490D8667C8F_element_21")>
--------------------
in for loop
row <selenium.webdriver.remote.webelement.WebElement (session="76251786298407fb119989d6a661ba50", element="87182FA69B721DFDF43E1490D8667C8F_element_22")>
--------------------
in for loop
row <selenium.webdriver.remote.webelement.WebElement (session="76251786298407fb119989d6a661ba50", element="87182FA69B721DFDF43E1490D8667C8F_element_23")>
--------------------
in for loop
row <selenium.webdriver.remote.webelement.WebElement (session="76251786298407fb119989d6a661ba50", element="87182FA69B721DFDF43E1490D8667C8F_element_24")>
--------------------
in for loop
row <selenium.webdriver.remote.webelement.WebElement (session="76251786298407fb119989d6a661ba50", element="87182FA69B721DFDF43E1490D8667C8F_element_25")>
--------------------
in for loop
row <selenium.webdriver.remote.webelement.WebElement 

...

in for loop
row <selenium.webdriver.remote.webelement.WebElement (session="76251786298407fb119989d6a661ba50", element="87182FA69B721DFDF43E1490D8667C8F_element_20")>
--------------------
in for loop
row <selenium.webdriver.remote.webelement.WebElement (session="76251786298407fb119989d6a661ba50", element="87182FA69B721DFDF43E1490D8667C8F_element_21")>
--------------------
in for loop
row <selenium.webdriver.remote.webelement.WebElement (session="76251786298407fb119989d6a661ba50", element="87182FA69B721DFDF43E1490D8667C8F_element_22")>
--------------------
in for loop
row <selenium.webdriver.remote.webelement.WebElement (session="76251786298407fb119989d6a661ba50", element="87182FA69B721DFDF43E1490D8667C8F_element_23")>
--------------------
in for loop
row <selenium.webdriver.remote.webelement.WebElement (session="76251786298407fb119989d6a661ba50", element="87182FA69B721DFDF43E1490D8667C8F_element_24")>
--------------------
in for loop
row <selenium.webdriver.remote.webelement.WebElement 

in for loop
row <selenium.webdriver.remote.webelement.WebElement (session="76251786298407fb119989d6a661ba50", element="87182FA69B721DFDF43E1490D8667C8F_element_20")>
--------------------
in for loop
row <selenium.webdriver.remote.webelement.WebElement (session="76251786298407fb119989d6a661ba50", element="87182FA69B721DFDF43E1490D8667C8F_element_21")>
--------------------
in for loop
row <selenium.webdriver.remote.webelement.WebElement (session="76251786298407fb119989d6a661ba50", element="87182FA69B721DFDF43E1490D8667C8F_element_22")>
--------------------
in for loop
row <selenium.webdriver.remote.webelement.WebElement (session="76251786298407fb119989d6a661ba50", element="87182FA69B721DFDF43E1490D8667C8F_element_23")>
--------------------
in for loop
row <selenium.webdriver.remote.webelement.WebElement (session="76251786298407fb119989d6a661ba50", element="87182FA69B721DFDF43E1490D8667C8F_element_24")>
--------------------
in for loop
row <selenium.webdriver.remote.webelement.WebElement 

in for loop
row <selenium.webdriver.remote.webelement.WebElement (session="76251786298407fb119989d6a661ba50", element="87182FA69B721DFDF43E1490D8667C8F_element_20")>
--------------------
in for loop
row <selenium.webdriver.remote.webelement.WebElement (session="76251786298407fb119989d6a661ba50", element="87182FA69B721DFDF43E1490D8667C8F_element_21")>
--------------------
in for loop
row <selenium.webdriver.remote.webelement.WebElement (session="76251786298407fb119989d6a661ba50", element="87182FA69B721DFDF43E1490D8667C8F_element_22")>
--------------------
in for loop
row <selenium.webdriver.remote.webelement.WebElement (session="76251786298407fb119989d6a661ba50", element="87182FA69B721DFDF43E1490D8667C8F_element_23")>
--------------------
in for loop
row <selenium.webdriver.remote.webelement.WebElement (session="76251786298407fb119989d6a661ba50", element="87182FA69B721DFDF43E1490D8667C8F_element_24")>
--------------------
in for loop
row <selenium.webdriver.remote.webelement.WebElement 

in for loop
row <selenium.webdriver.remote.webelement.WebElement (session="76251786298407fb119989d6a661ba50", element="87182FA69B721DFDF43E1490D8667C8F_element_20")>
--------------------
in for loop
row <selenium.webdriver.remote.webelement.WebElement (session="76251786298407fb119989d6a661ba50", element="87182FA69B721DFDF43E1490D8667C8F_element_21")>
--------------------
in for loop
row <selenium.webdriver.remote.webelement.WebElement (session="76251786298407fb119989d6a661ba50", element="87182FA69B721DFDF43E1490D8667C8F_element_22")>
--------------------
in for loop
row <selenium.webdriver.remote.webelement.WebElement (session="76251786298407fb119989d6a661ba50", element="87182FA69B721DFDF43E1490D8667C8F_element_23")>
--------------------
in for loop
row <selenium.webdriver.remote.webelement.WebElement (session="76251786298407fb119989d6a661ba50", element="87182FA69B721DFDF43E1490D8667C8F_element_24")>
--------------------
in for loop
row <selenium.webdriver.remote.webelement.WebElement 

WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=114.0.5735.199)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x0081A813+48355]
	(No symbol) [0x007AC4B1]
	(No symbol) [0x006B5358]
	(No symbol) [0x006A7D96]
	(No symbol) [0x006A7AB9]
	(No symbol) [0x006B67C0]
	(No symbol) [0x0070C4D8]
	(No symbol) [0x006FA536]
	(No symbol) [0x006D82DC]
	(No symbol) [0x006D93DD]
	GetHandleVerifier [0x00A7AABD+2539405]
	GetHandleVerifier [0x00ABA78F+2800735]
	GetHandleVerifier [0x00AB456C+2775612]
	GetHandleVerifier [0x008A51E0+616112]
	(No symbol) [0x007B5F8C]
	(No symbol) [0x007B2328]
	(No symbol) [0x007B240B]
	(No symbol) [0x007A4FF7]
	BaseThreadInitThunk [0x75B87D59+25]
	RtlInitializeExceptionChain [0x7756B79B+107]
	RtlClearBits [0x7756B71F+191]


In [ ]:
# Quit the driver
driver.quit()

In [ ]:
# load and check the scraped data
data = pd.read_csv('data/data.csv')

In [ ]:
data

,Ticker,Instrument,Nombre de titres,Cours (MAD),Capitalisation,date
0,MDP,MED PAPER,4 783 823,"28,98","138 635 190,54",26 décembre 2022
1,ALM,ALUMINIUM DU MAROC,465 954,"1 420,00","661 654 680,00",26 décembre 2022
2,AGM,AGMA,200 000,"6 050,00","1 210 000 000,00",26 décembre 2022
3,NEJ,AUTO NEJMA,1 023 264,"1 817,00","1 859 270 688,00",26 décembre 2022
4,GAZ,AFRIQUIA GAZ,3 437 500,"4 525,00","15 554 687 500,00",26 décembre 2022
...,...,...,...,...,...,...
6511,DYT,DISTY TECHNOLOGIES,998 110,"180,00","179 659 800,00",2 mai 2023
6512,AKT,AKDITAL,12 666 670,"298,00","3 774 667 660,00",2 mai 2023
6513,DLM,DELATTRE LEVIVIER MAROC,1 750 000,"40,00","70 000 000,00",2 mai 2023
6514,OUL,OULMES,1 980 000,"1 322,00","2 617 560 000,00",2 mai 2023


<hr>

### Scrape data related to shareholders